In [ ]:
#Won't work now b/c I deleted the AI Platform stuff so I wouldn't get charged

!gsutil -m cp gs://kams-bucket/model/cats_vs_dogs.h5 .

In [ ]:
import tensorflow as tf

model =tf.keras.models.load_model('cats_vs_dogs.h5')
model.summary()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
classes = ['Cat', 'Dog']

def display_pred(image_path):
    plt.imshow(plt.imread(image_path))
    
    x = tf.keras.preprocessing.image.load_img(image_path, target_size=(128, 128))
    
    x = tf.keras.preprocessing.image.img_to_array(x)
    x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
    x = np.expand_dims(x, axis=0)
    
    pred = model.predict(x)[0]
    
    #Number is close to 1 if a dog - number is close to 0 if cat
    plt.title('Pred: {}'.format(classes[int(pred > 0.5)]))
    plt.show()
    
    print(pred)

In [ ]:
images = ['images/cat1.jpg', 'images/cat2.jpg', 'images/dog1.jpg']

display_pred(images[2])

In [ ]:
%%writefile prediction.py

import tensorflow as tf
import numpy as np
import os
import base64

MODEL_NAME = 'cats_vs_dogs.h5'
CLASS_NAMES = ['Cat', 'Dog']

class CatsVsDogsPrediction:
    def __init__(self, model):
        self.model = model
    
    def _preprocess(self, instances, size=128):
        num_examples = len(instances)
        x_batch = np.zeros((num_examples, size, size, 3))
        for i in range(num_examples):
            #coding string into numpy array
            x = np.array(bytearray(base64.b64decode(instances[i])))
            x = np.reshape(x, (size, size, 3))
            x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
            x_batch[i] = x
        return x_batch
    
    def _postprocess(self, preds):
        results = []
        for i, pred in enumerate(preds):
            p = np.squeeze(pred)
            results.append({
                'index': i, 
                'class_name': CLASS_NAMES[int(p > 0.5)],
                'raw_value': '{:.4f}'.format(p)
            })
        return results
        
    def predict(self, instances, **kwargs):
        if 'size' in kwargs:
            size = int(kwargs.get('size'))
        else:
            size = 128
        # Preprocess
        x_batch = self._preprocess(instances, size)
        # Predict
        preds = self._model.predict(x_batch)
        # Post process
        results = self._postprocess(preds)
        return results
    
    @classmethod
    def from_path(cls, model_dir):
        model = tf.keras.models.load_model(os.path.join(model_dir, MODEL_NAME))
        return cls(model)

In [ ]:
%%writefile setup.py
from setuptools import setup

setup(
    name='cats_vs_dogs', 
    version='0.0.1',
    include_package_data=False,
    scripts=['prediction.py']
)

In [ ]:
!python3 setup.py sdist --formats=gztar

In [ ]:
!gsutil cp dist/cats_vs_dogs-0.0.1.tar.gz gs://kams-bucket/dist/

In [ ]:
#had trouble importing my own model --> in AI Platform, when adding a model, had trouble with creating the version
from googleapiclient import discovery
from PIL import Image
import os
import base64

In [ ]:
service = discovery.build('ml', 'vl', cache_discovery=False)

In [ ]:
def get_pred_from_model(body, project_name, model_name):
    service.projects().predict(
        name='projects/{}/models/{}'.format(project_name, model_name),
        body=body
    )

In [ ]:
project_name = 'rhyme-269417' #mine would be different
model_name = 'cats_vs_dogs'

In [ ]:
instances = []
size = 128

for image in images:
    img = Image.open(image)
    img = img.resize((size, size), Image.ANTIALIAS)
    
    instances.append(
        base64.b64encode(img.tobytes()).decode()
    )
    img.close()

In [ ]:
body = {
    'instances': instances,
    'size': size
}

In [ ]:
response = get_pred_from_model(body, project_name, model_name)